# Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import gensim
from gensim.models import Word2Vec
from wordcloud import WordCloud

from nltk.stem import PorterStemmer
from textblob import TextBlob

nltk.download("vader_lexicon")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

# Machine Learning Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("df.csv")
df.head()

,ReviewText,Review,ReviewLength,PlayHours,DatePosted,Language,Positive,Negative,Neutral,Compound,Sentiment,AspectTerm
0,bad game ever even make purchase man i am buy ...,Not Recommended,96,53.1 hrs on record,Posted: 14 November,en,0.000,0.350,0.650,-0.7906,Negative,"['price', 'gameplay']"
1,game rigged casino,Not Recommended,28,136.4 hrs on record,Posted: November 13,en,0.000,0.556,0.444,-0.3612,Negative,['gameplay']
2,start good begin suck greatly couple hour like...,Not Recommended,440,39.0 hrs on record,Posted: November 13,en,0.285,0.070,0.645,0.9125,Negative,"['price', 'game functionality/bug', 'gameplay']"
3,someone force play kbdmouse due controller inc...,Not Recommended,379,71.0 hrs on record,Posted: November 13,en,0.045,0.220,0.734,-0.8519,Negative,"['visual', 'price', 'game functionality/bug', ..."
4,help regard direct x error,Recommended,28,62.5 hrs on record,Posted: November 10,en,0.365,0.365,0.270,0.0000,Negative,['game functionality/bug']


## Encoding Target

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Sentiment'] = encoder.fit_transform(df['Sentiment'])
y = df['Sentiment']

## Word Embbeding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Text embedding using TF-IDF
vectorizer = TfidfVectorizer()
# vectorizer = CountVectorizer()
X = df['ReviewText']
X = vectorizer.fit_transform(X)

## Machine Learning Model

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True)
}

results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Predict on test data

    # Calculate weighted metrics
    accuracy = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted')
    weighted_recall = recall_score(y_test, y_pred, average='weighted')

    # Calculate F1 score per class
    f1_negative = f1_score(y_test, y_pred, labels=[0], average=None).mean()
    f1_neutral = f1_score(y_test, y_pred, labels=[1], average=None).mean()
    f1_positive = f1_score(y_test, y_pred, labels=[2], average=None).mean()

    # Append results
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Weighted Precision": weighted_precision,
        "Weighted Recall": weighted_recall,
        "F1-Negative": f1_negative,
        "F1-Neutral": f1_neutral,
        "F1-Positive": f1_positive
    })

# Create a DataFrame for comparison
results_df = pd.DataFrame(results)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:32:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
results_df

,Model,Accuracy,Weighted Precision,Weighted Recall,F1-Negative,F1-Neutral,F1-Positive
0,Decision Tree,0.795742,0.794715,0.795742,0.848061,0.614815,0.743802
1,Naive Bayes,0.746301,0.699813,0.746301,0.836256,0.000000,0.614251
2,Logistic Regression,0.845904,0.844096,0.845904,0.891758,0.491228,0.829009
3,Random Forest,0.842656,0.845304,0.842656,0.887417,0.678788,0.785664
4,XGBoost,0.848791,0.847810,0.848791,0.891755,0.649087,0.811868
5,SVM,0.845182,0.853690,0.845182,0.891052,0.465753,0.824087
